# Notebook 16: Expectation Maximization in practice


## Learning Goal 
The goal of this notebook is to gain intuition for Expectation Maximization using a simple example involving coin tosses.

## Overview

In Section XIV, we introduce Expectation-Maximization (EM) as a practical way to perform maximum likelihood estimation (MLE) even when some of the data is hidden (i.e in the presence of latent or hidden variables). To better understand EM, in this short notebook we'll explore a very simple coin-tossing example adapted from [Do and Batzoglou, Nat. Biotechnol. (2008)](https://www.nature.com/articles/nbt1406). 

Suppose that we are given two coins A and B with unkown bias $\theta_A$ and $\theta_B$, respectively. Our goal is to estimate the bias vector $\boldsymbol{\theta}= (\theta_A, \theta_B)$ from the outcomes of the following experiment: 

<blockquote> 
First choose one coin at random. Then toss the selected coin 10 times independently and record the number of heads observed. Repeat this procedure 5 times.
</blockquote>

Formally, let $z_i\in\{A,B\}$ be the coin selected in experiment $i$ and $x_i\in\{0,1,\cdots 10\}$ be the number heads recorded by tossing $z_i$ 10 times. Since we conduct $n=5$ such experiments, we can summarize the outcomes of these 50 tosses by two vectors: $\boldsymbol{x}=(x_1,x_2\cdots, x_5)$ and $\boldsymbol{z}=(z_1,z_2,\cdots, z_5)$.




### Exercise 1: What if we know everything?

 * Consider first the case where we have complete knowledge of the experiment, namely, both $\boldsymbol{x}$ and $\boldsymbol{z}$ are known. How would you intuitively estimate the biases of the two coins  $\boldsymbol{\theta}= (\theta_A, \theta_B)$ ?
 
 * What's the likelihood of observing the complete outcomes of these experiments? In other words, what is $P(\boldsymbol{x},\boldsymbol{z}| n,\boldsymbol{\theta} )$? You may assume this is a Bernoulli trial. Namely, every time coin A(B) is tossed, we have, with probability $\theta_A$($\theta_B$), that the outcome is heads.
 
 * What's the Maximum Likelihood Estimator (MLE)? Is this consistent with your intuition? 


### Answers to Exercise 1:

<ul>
    <li>Probability that for $N$ trials, $N_H$ heads are obtained goes according to the binomial distribution: $$P(N_H | N) = \binom{N}{N_H} \theta^{N_H} (1-\theta)^{N-N_H}$$, where $\theta$ is the probability/bias of obtaining a head. Therefore, the mean of the distribution could give us an estimate of the bias of the coin, which would be $N_H/N$(mean of a binomial distribution).
    </li>
    <li>The total probability of observing the results of coin toss experiments is the product of the individual probabilities by virtue of the two coin tosses being independent of one another:
    $$P(x,z|N,\theta) = \prod_i P_{binomial}(N_i|N,\theta_A)^{1-z_i} P_{binomial}(N_i|N,\theta_B)^{z_i}$$ ;$z_i=0$ for coin A and $z_i=1$ for coin B, $N_i$ the number of heads observed in the trial $i$ such that $\sum_{i}{N_i} = N$.
    </li>
    <li> Maximizing the above log-probability w.r.t $\theta$, one would get: 
    $$
    \log{P} \sim \sum_i ( N_H^i \log(\theta) + (N-N_H^i)\log{(1-\theta))$$
    
    $$\frac{\partial \log{P}}{\partial \theta} \sim \sum_i( \frac{N_H^i}{\theta} - \frac{N-N_H^i}{1-\theta})$$ 
   
    $$\frac{\partial \log{P}}{\partial \theta} = 0 $$
    $$\implies N_H - \theta N_{total} = 0$$
    $$\implies \theta = \frac{N_H}{N_{total}}$$
    $$MLE_{A} = \frac{N_A^H}{N_A}$$ and $$MLE_{B} = \frac{N_B^H}{N_B}$$ ,where $N_A^H + N_B^H = N_H$, which is consistent with the answer to the first question.
    
   </li>
</ul>

## Comparing MLE and EM

To test your answer, let's do some numerics! We will compare the MLE estimates of biases with an Expectation Maximization procedure where we do not know ${\bf z}$. The following code computes our best guess for the biases using MLE -- assuming we know the identity of the coin used -- and compares it estimates arrived at using an EM procedure where we have no knowledge about which coin was being tossed (though we know the same coin was tossed 10 times).

In [ ]:
import numpy as np
from scipy.special import comb
import math


def compute_likelihood(obs, n, pheads): # No surprise, it's Binomial!!!

    likelihood = comb(n, obs, exact=True)*(pheads**obs)*(1.0-pheads)**(n-obs)

    return likelihood

# generate experiments
'''num_coin_toss = 10 # each experiment contains num_coin_toss tosses
num_exp = 5  # we perform 5 such experiments
theta_A_true = 0.9
theta_B_true = 0.4
coin_choice = np.zeros(num_exp) # initialize: 0 for A and 1 for B
head_counts = np.zeros(num_exp)'''

guess_theta_A_list = []
guess_theta_B_list = []
theta_A_list = []
theta_B_list = []
MLE_A_list = []
MLE_B_list = []

for trial in range(50000):
    print('{}/{}'.format(trial, 50000), end='\r')
    
    num_coin_toss = 10 
    num_exp = 5
    theta_A_true = 0.8 
    theta_B_true = 0.4
    coin_choice = np.zeros(num_exp) # initialize: 0 for A and 1 for B
    head_counts = np.zeros(num_exp)

    # MLE 
    MLE_A = 0.0
    MLE_B = 0.0

    # generate the outcomes of experiment
    for i in np.arange(num_exp):

        if np.random.randint(2) == 0: # coin A is selected
            head_counts[i] = np.random.binomial(num_coin_toss , theta_A_true, 1) # toss coin A num_coin_toss times
            MLE_A = MLE_A +  head_counts[i] # add the number of heads observed to total headcounts 

        else: # coin B is selected 
            head_counts[i] = np.random.binomial(num_coin_toss , theta_B_true, 1) # toss coin B num_coin_toss times
            coin_choice[i] = 1  # record the selection of coin B during experiment i 
            MLE_B = MLE_B +  head_counts[i] # add the number of heads observed to total headcounts 

    tail_counts = num_coin_toss - head_counts

    try:
        # MLE is merely the proportion of heads for each coin toss
        MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice))*num_coin_toss)
        MLE_B = MLE_B / (np.count_nonzero(coin_choice)*num_coin_toss)
    except:
        continue

    MLE_A_list.append(MLE_A)
    MLE_B_list.append(MLE_B)
    
    # initialize the pA(heads) and pB(heads), namely, coin biases
    pA_heads = np.zeros(100); 
    pB_heads = np.zeros(100); 

    # Draw the guesses randomly from uniform distribution
    #pA_heads[0] = np.random.uniform()
    #pB_heads[0] = np.random.uniform()
    #print('*'*20)
    #print('Initial guesses: pA = {pA:.3f} pB = {pB:.3f}'.format(pA=pA_heads[0], pB=pB_heads[0]))

    pA_heads[0] = 0.60 # initial guess
    pB_heads[0] = 0.50 # initial guess

    # E-M begins!
    epsilon = 0.001   # error threshold
    j = 0 # iteration counter
    improvement = float('inf')

    while (improvement > epsilon):

        expectation_A = np.zeros((num_exp,2), dtype=float) 
        expectation_B = np.zeros((num_exp,2), dtype=float)

        for i in np.arange(min(len(head_counts),len(tail_counts))):

            eH = head_counts[i]
            eT = tail_counts[i]

            # E step:
            lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
            lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])
            weightA = lA / (lA + lB) # Probability that coin A is chosen
            weightB = lB / (lA + lB) # Probability that coin B is chosen

            expectation_A[i] = weightA*np.array([eH, eT]) # Number of heads/tails assigned to A
            expectation_B[i] = weightB*np.array([eH, eT]) # Number of heads/tails assigned to B

        # M step
        theta_A = np.sum(expectation_A, axis = 0)[0] / np.sum(expectation_A) 
        theta_B = np.sum(expectation_B, axis = 0)[0] / np.sum(expectation_B) # MLE estimate 
        #N_tot_A = np.sum(expectation_A)
        #N_tot_B = np.sum(expectation_B)
        #N_heads_A = np.sum(expectation_A, axis = 0)[0]
        #N_heads_B = np.sum(expectation_B, axis = 0)[0]
        #try:
        #    theta_A = find_root(mu=0.83, sigma=1, N_heads=N_heads_A, N_tot=N_tot_A)
        #    theta_B = find_root(mu=0.37, sigma=1, N_heads=N_heads_B, N_tot=N_tot_B)
        #except: 
        #    print('Something happened')
        #    continue
        
        #print('At iteration %d, theta_A = %2f,  theta_B = %2f' % (j, theta_A, theta_B))

        pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A)); 
        pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B)); 
        #pA_heads[j+1] = theta_A
        #pB_heads[j+1] = theta_B

        improvement = max( abs(np.array([pA_heads[j+1],pB_heads[j+1]]) - np.array([pA_heads[j],pB_heads[j]]) ))
        j = j+1

    # END of E-M, print the outcome

    #print('E-M converges at iteration %d' %j)
    #print('RESULT:')
    #print('E-M: theta_A = %2f,  theta_B = %2f' % (theta_A, theta_B))
    #print('MLE with complete data: theta_A = %2f,  theta_B = %2f' % (MLE_A, MLE_B))
    
    guess_theta_A_list.append(pA_heads[0])
    guess_theta_B_list.append(pB_heads[0])
    theta_A_list.append(theta_A)
    theta_B_list.append(theta_B)
print ('done')
# generate the outcomes of experiment
'''for i in np.arange(num_exp):
    
    if np.random.randint(2) == 0: # coin A is selected
        head_counts[i] = np.random.binomial(num_coin_toss , theta_A_true, 1) # toss coin A num_coin_toss times
        MLE_A = MLE_A +  head_counts[i] # add the number of heads observed to total headcounts 
    
    else: # coin B is selected 
        head_counts[i] = np.random.binomial(num_coin_toss , theta_B_true, 1) # toss coin B num_coin_toss times
        coin_choice[i] = 1  # record the selection of coin B during experiment i 
        MLE_B = MLE_B +  head_counts[i] # add the number of heads observed to total headcounts 
    
tail_counts = num_coin_toss - head_counts


# MLE is merely the proportion of heads for each coin toss
MLE_A = MLE_A / ((num_exp - np.count_nonzero(coin_choice))*num_coin_toss)
MLE_B = MLE_B / (np.count_nonzero(coin_choice)*num_coin_toss)



# initialize the pA(heads) and pB(heads), namely, coin biases
pA_heads = np.zeros(100); 
pB_heads = np.zeros(100); 

pA_heads[0] = 0.60 # initial guess
pB_heads[0] = 0.5 # initial guess

# E-M begins!
epsilon = 0.001   # error threshold
j = 0 # iteration counter
improvement = float('inf')

while (improvement > epsilon):
    
    expectation_A = np.zeros((num_exp,2), dtype=float) 
    expectation_B = np.zeros((num_exp,2), dtype=float)
    
    for i in np.arange(min(len(head_counts),len(tail_counts))):
        
        eH = head_counts[i]
        eT = tail_counts[i]
        
        # E step:
        lA = compute_likelihood(eH, num_coin_toss, pA_heads[j])
        lB = compute_likelihood(eH, num_coin_toss, pB_heads[j])
        
        weightA = lA / (lA + lB)
        weightB = lB / (lA + lB)
              
        expectation_A[i] = weightA*np.array([eH, eT])
        expectation_B[i] = weightB*np.array([eH, eT])

  
    # M step
    theta_A = np.sum(expectation_A, axis = 0)[0] / np.sum(expectation_A) 
    theta_B = np.sum(expectation_B, axis = 0)[0] / np.sum(expectation_B) 

    print('At iteration %d, theta_A = %2f,  theta_B = %2f' % (j, theta_A, theta_B))
    
    pA_heads[j+1] = sum(expectation_A)[0] / sum(sum(expectation_A)); 
    pB_heads[j+1] = sum(expectation_B)[0] / sum(sum(expectation_B)); 

    improvement = max( abs(np.array([pA_heads[j+1],pB_heads[j+1]]) - np.array([pA_heads[j],pB_heads[j]]) ))
    j = j+1

# END of E-M, print the outcome

print('E-M converges at iteration %d' %j)
print('RESULT:')
print('E-M: theta_A = %2f,  theta_B = %2f' % (theta_A, theta_B))
print('MLE with complete data: theta_A = %2f,  theta_B = %2f' % (MLE_A, MLE_B))'''

### Exercise 2

 * How fast does EM converge? Is the converged result close to what you'd get from MLE? 
 
 * Following Exercise 1, what's the objective function we're optimizing in the E-step? Does this function have a *unique global maximum*? 
 
 * Compare both the results of MLE and EM to the actual bias (i.e. *theta_A_true*  and *theta_B_true* in the snippet above), comment on their performance.
 


### Solutions:

<ul>
    <li>Depending on the random initialization, EM can converge in 7-8 steps to ~18-20 steps. The result, as to how far it is or not from the one obtained via MLE also depends on the random initialization. The closer the initialization is to the true biases, the longer it takes via EM. This is probably because of small steps taken.
    </li>
    <li>In the E-step, we are tweaking the bias in every step starting with an initial guess so as to maximise the log-probability. The problem does not have a global maximum; different initializations leading to different local maxima.
    </li>
    <li> The closer the initial condition to the bias, the better EM performs. Otherwise, they both are fairly accurate. Therefore, overall MLE seems to have more accurate results. I think this is because in MLE, we know which coin is tossed, thereby giving us more information.

## Final remarks: a few practical tricks

From Exercise 2 and Section XIV, we know that the E-M algorithm often approximates the MLE even in the presence of latent (hidden variables). Like with most optimization methods for nonconcave functions, E-M only guarantees convergence to a local maximum of the objective function. For this reason, its performance can be boosted by running the EM procedure starting with multiple initial parameters. 

### Exercise 3

* Now instead of having a fixed initial guess of coin biases (i.e. *pA_heads[0]* and *pB_heads[0]* in the snippet), draw these values uniformly at random from $[0,1]$ and run the E-M algorithm. Repeat this twenty times and report what you observed. What's the best initial guess that gives the closest estimate to the true parameters?

* As we discussed in Section X (LinReg), **Maximum a posteriori (MAP)** estimation differs from MLE in that it employs an augmented objective function which incorporates a prior distribution over the quantities we want to estimate, and the prior distribution can be think of as a regularizer for the objective fuction used in MLE. Here we will explore how to extend E-M to MAP estimation. 

  (1) First derive the MAP estimate for the one-coin-flipping example, namely,
  $$
  \hat{{\theta}}_{MAP}(\boldsymbol{x}) = \arg\max_{\theta\in[0,1]} \log P(\boldsymbol{x}|n,{\theta} ) + \log P({\theta}),
  $$
  where 
  $$P(\boldsymbol{x}|n,{\theta}) = \prod_{i=1}^{10} \text{Binomial} (x_i|n,\theta)$$
  
  $$P({\theta})=\mathcal{N}(\theta|\mu, \sigma)$$
  
  (2) Based on (1), now modify the E-M snippet above to incorporate this prior distribution into the **M-step**. Comment on the performance. For the prior choice, try $P(\boldsymbol{\theta})=\mathcal{N}(\theta_A|0.83, 1)\mathcal{N}(\theta_B|0.37, 1)$.

### Solutions:

1. When uesses drawn from an uniform distribution between [0, 1], the final answers are reasonably accurate only when the initial guess for the bias is close to the true bias. Similarly, we get the worst results for the initial guesses that are furthest away from the true biases. Out of 50000 trials, we get: <br>
  Best guess: <br>
  $p_A = 0.8013$ <br>
  $p_B = 0.3986$ 
  
  Worst guess: <br>
  $p_A = 0.0002$ <br>
  $p_B = 0.9896$
  
  Comparing these biases with the true biases, $p_A = 0.8$ and $p_B = 0.4$, we understand that we get the best estimates for the initial conditions that are closest to the actual values. Below, histograms are shown for the outcomes of the 50000 trials with EM and pure MLE estimators with both coins: <br>
 
  
  For both coins, although we see some accumulation of trials near right bias ($0.8$ and $0.4$ for A and B, respectively), the distribution is pretty much flat around $\theta$ between $0.4$ and $0.9$. Our bias estimates with EM algorithm get close to the real biases for around $\sim 40 \%$ of the time. But from the MLE results (bottom), we can observe that the trials are peaked at the actual biases, therefore we can conclude that pure MLE estimation performs better in general. We also observe that the results for EM algorithm is <b>worse</b> compared to the case with the fixed initial guesses.
  
2. The prior is: $P(\theta)=e^{-\frac{\theta - \mu}{\sigma^2}}$.

To minimize:
$\log{P} = \sum_{i=1} (N_H^i\log{\theta} + (N-N_H^i)\log{(1-\theta)}) - \frac{(\theta - \mu)^2}{2\sigma^2} $, 

Maximizing $\log{P}$: 
$\frac{1}{\theta(1-\theta)}\sum_{i=1} (N_H^i - N\theta) = \frac{\theta - \mu}{\sigma^2}$ <br>
Using $\sum_{i=1}N_H^i = N_H$ and $\sum_{i=1} N = N_{tot}$.

$\frac{1}{\theta(1-\theta)}(N_H - N_{total}\theta) = \frac{\theta - \mu}{\sigma^2}$ <br>

Solution to this equation for $\theta$ gives us the MAP estimate.